In [1]:
import sys
import warnings
import torch
import torch_geometric.transforms as T
from utils import Complete
from dataset_no_rdkit import QM9Dataset
from trainer import Trainer
from model import RGNN


In [2]:
warnings.filterwarnings("ignore")
task = 2
option = {
    'train_epoch': 2,
    'train_batch': 2,
    'task': task,  # [0~11]
    'lr': 0.001,
    'lr_scheduler_patience': 3,
    'parallel': False,
    'cuda_devices': [0, 1, 2, 3],  # works when parallel=True
    'early_stop_patience': 10,  # -1 for no early stop
}


class SpecifyTarget(object):
    def __call__(self, data):
        data.y = data.y[option['task']].view(-1)
        return data


print('Load Dataset...')
transform = T.Compose([SpecifyTarget(), Complete(), T.Distance(norm=True)])
path = '../QM9_dataset_moleculenet'
dataset = QM9Dataset(root=path, transform=transform).shuffle()

print('Split the dataset...')
one_tenth = len(dataset) // 10000
test_dataset = dataset[: one_tenth]
valid_dataset = dataset[one_tenth: one_tenth * 2]
train_dataset = dataset[one_tenth * 2: one_tenth * 3]

Load Dataset...
Split the dataset...


In [3]:
print('Training init...')
model = RGNN()
trainer = Trainer(option, model, \
                  train_dataset, valid_dataset, test_dataset)

Training init...
	train_epoch:2
	train_batch:2
	task:2
	lr:0.001
	lr_scheduler_patience:3
	parallel:False
	cuda_devices:[0, 1, 2, 3]
	early_stop_patience:10
save id: 06b1
train set num:13 valid set num:13 test set num: 13
Total Parameters:2865729


In [4]:
trainer.train()

Training start...


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

	batch 0 training loss: 0.50139 Time elapsed 0.00 hrs (0.0 mins)
Epoch:0 trn_loss:0.18064 val_loss:0.03549 Time elapsed 0.00 hrs (0.1 mins)
Model saved at epoch 0


 50%|██████████████████████████████████████████                                          | 1/2 [00:08<00:08,  8.32s/it]

	batch 0 training loss: 0.25100 Time elapsed 0.00 hrs (0.2 mins)
Epoch:1 trn_loss:0.09517 val_loss:0.17754 Time elapsed 0.00 hrs (0.3 mins)


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:16<00:00,  8.17s/it]


Model saved at epoch 1


In [5]:
trainer.load_best_ckpt()

The best ckpt is D:\Jupyter\RGNN\src_v7.3_1e-4lr\ckpt\ckpts_task2_06b1/task2_0_0.18064_0.03549.ckpt
Ckpt loading: D:\Jupyter\RGNN\src_v7.3_1e-4lr\ckpt\ckpts_task2_06b1/task2_0_0.18064_0.03549.ckpt


In [6]:
trainer.valid_iterations(mode='test')

Test loss: 0.03016


0.03015950508415699